In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import get_ae_data_acfiles_v1

TEST_NAME = 'p4677'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4677/'

In [ ]:
# Mech data
with h5py.File(DATA_DIR+TEST_NAME+'_mechdata.hdf5', 'r') as f:
  time_mech = f.get('time')[:]
  mu = f.get('mu')[:]
  t2f = f.get('t2f')[:]
  tfail = f.get('tfail')[:]
print('tfail', tfail)

# AE data info
Syncfile = loadmat(DATA_DIR+'p4677_AcTimeVector_Run2.mat')
acsettings = loadmat(DATA_DIR+'p4677_run2_AE.mat')
ACfile_DIR = DATA_DIR+'run2/AcFiles/'

# number of superframes per file
numSFpfile = int(acsettings['numFrames'][0]/2)
# number of WF per superframe and per channel
numWFpSFpCH = int(acsettings['numAcqs'][0])
# number of WF per file and per channel
numWFpfilepCH = numSFpfile * numWFpSFpCH
# number of channels
numCHR = len(acsettings['channels2save'][0])
# waveform length
WFlength = int(acsettings['Nsamples'][0])
# true number of points per file
TruenumPointspfilepCH = numWFpfilepCH * WFlength

print('numSFpfile', numSFpfile)
print('numWFpSFpCH', numWFpSFpCH)
print('numWFpfilepCH', numWFpfilepCH)
print('numCHR', numCHR)
print('WFlength', WFlength)
print('TruenumPointspfilepCH', TruenumPointspfilepCH)

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(16,8), sharex=True)
fig.subplots_adjust(hspace=0.0)

ax[-2].plot(time_mech, mu, 'b-', linewidth=1.0)
ax[-2].set_xlabel('Time [s]')
ax[-2].set_ylabel('Friction coefficient')
ax[-2].grid()
ax[-2].xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.3f'))

ax[-1].plot(time_mech, t2f, 'k-', linewidth=1.0)
ax[-1].plot(tfail, np.zeros_like(tfail), 'ro')
ax[-1].set_xlabel('Time [s]')
ax[-1].set_ylabel('T2F [s]')
ax[-1].grid()
ax[-1].xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.3f'))

In [ ]:
start_file_number = 40
end_file_number = 485

wf_ae, time_ae, time_ae_seg = get_ae_data_acfiles_v1(ACfile_DIR, acsettings, Syncfile, start_file_number)
print(wf_ae.shape, time_ae.shape, time_ae_seg.shape)
print(time_ae)

fig = plt.figure(figsize=(16,4))
ax = plt.gca()
plt.plot(time_ae, wf_ae[:,0], 'k-')
plt.xlabel('Time [s]')
plt.ylabel('AE')
plt.grid()

In [ ]:
wf_ae, time_ae, time_ae_seg = get_ae_data_acfiles_v1(ACfile_DIR, acsettings, Syncfile, end_file_number)
print(wf_ae.shape, time_ae.shape, time_ae_seg.shape)
print(time_ae)

fig = plt.figure(figsize=(16,4))
ax = plt.gca()
plt.plot(time_ae, wf_ae[:,0], 'k-')
plt.xlabel('Time [s]')
plt.ylabel('AE')
plt.grid()

In [ ]:
with h5py.File(DATA_DIR+TEST_NAME+'_ae_data.hdf5', "w") as f:
  dset_ae = f.create_dataset('ae', (0, numCHR), maxshape=(None, numCHR), dtype='<i2')
  dset_time = f.create_dataset('time', (0,), maxshape=(None,), dtype='<f8')
  dset_time_seg = f.create_dataset('time_seg', (0,), maxshape=(None,), dtype='<f8')
  
  dset_time_seg.attrs['WFlength'] = WFlength
  dset_time_seg.attrs['numWFpfilepCH'] = numWFpfilepCH
  dset_time_seg.attrs['TruenumPointspfilepCH'] = TruenumPointspfilepCH
  
  for filenumber in range(start_file_number, end_file_number):
    dset_ae.resize(dset_ae.shape[0]+TruenumPointspfilepCH, axis=0)
    dset_time.resize(dset_time.shape[0]+TruenumPointspfilepCH, axis=0)
    dset_time_seg.resize(dset_time_seg.shape[0]+numWFpfilepCH, axis=0)
    
    wf_ae, time_ae, time_ae_seg = get_ae_data_acfiles_v1(ACfile_DIR, acsettings, Syncfile, filenumber)
    
    print('  writing ae file', filenumber, time_ae[0], time_ae[-1], wf_ae.shape, time_ae.shape, time_ae_seg.shape)
    
    dset_ae[-TruenumPointspfilepCH:,:] = wf_ae
    dset_time[-TruenumPointspfilepCH:] = time_ae
    dset_time_seg[-numWFpfilepCH:] = time_ae_seg